In [2]:
# %pip install python-binance pandas
# %pip uninstall numpy -y
# %pip install "numpy<=2.2"
# %pip install numba pandas-ta

In [3]:
import pandas as pd
import pandas_ta as ta

In [4]:
from binance.client import Client
# Connection au client publique  de binance us
client = Client(tld='us')

In [5]:

currency = "BTCUSDT" # on prend une monnaie par exemple le bitcoin
interval = Client.KLINE_INTERVAL_6HOUR # intervalle de temps
start_str = "1 Jan, 2023" # date de début

# Récupération des données historiques de chandeliers (klines)
klines = client.get_historical_klines(currency, interval, start_str)

C:\Users\etulyon1\AppData\Local\Temp\ipykernel_9884\3063244892.py:6: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  klines = client.get_historical_klines(currency, interval, start_str)


In [ ]:
data = pd.DataFrame(klines, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
                                     'Close time', 'Quote asset volume', 'Number of trades',
                                        'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])

print(data.head())

       Open time            Open            High             Low  \
0  1672531200000  16524.41000000  16554.60000000  16500.63000000   
1  1672552800000  16532.16000000  16555.37000000  16497.67000000   
2  1672574400000  16555.97000000  16595.36000000  16536.43000000   
3  1672596000000  16583.67000000  16628.20000000  16583.67000000   
4  1672617600000  16619.93000000  16704.57000000  16549.68000000   

            Close        Volume     Close time Quote asset volume  \
0  16527.99000000   63.61115000  1672552799999   1051262.15231394   
1  16550.51000000   85.88487400  1672574399999   1419720.54247976   
2  16580.41000000   71.46302200  1672595999999   1183663.58698003   
3  16617.57000000   88.09700500  1672617599999   1462898.25611779   
4  16648.93000000  147.63305400  1672639199999   2454235.85600287   

   Number of trades Taker buy base asset volume Taker buy quote asset volume  \
0              1923                 29.01044700              479525.69463235   
1              2

In [ ]:
# Nettoyage et préparation des données
data['Open time'] = pd.to_datetime(data['Open time'], unit='ms')
for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# On prend que les données qui vont nous servir pour le MLOps
data = data[['Open time', 'Open', 'High', 'Low', 'Close', 'Volume']]

data['VolumeChange'] = data['Volume'].pct_change()
# RSI : Indice de force relative
data['RSI'] = ta.rsi(data['Close'], length=14)

# ATR : moyenne de la plage vraie
data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)

# Calcul de la volatilité et de la cible de volatilité
data['Log_Return'] = (data['Close'] / data['Close'].shift(1))

data['volatility'] = data['Log_Return'].rolling(window=4).std()

data['Target_Volatility'] = data['volatility'].shift(-4)

data = data.dropna()


print(data[['Open time', 'High', 'Low', 'ATR','volatility', 'Target_Volatility']].head())

# Si vous voulez sauvegarder les données dans un fichier CSV
# data.to_csv(f'binance_{currency}_6h.csv', index=False)

             Open time      High       Low         ATR  volatility  \
26 2023-01-07 12:00:00  16946.54  16910.00  103.908571    0.004062   
27 2023-01-07 18:00:00  16951.60  16928.19   98.158673    0.000882   
28 2023-01-08 00:00:00  16954.64  16913.17   94.109483    0.000753   
29 2023-01-08 06:00:00  16964.00  16924.27   90.225234    0.000767   
30 2023-01-08 12:00:00  17020.14  16914.80   91.304860    0.000754   

    Target_Volatility  
26           0.000754  
27           0.006049  
28           0.005747  
29           0.005127  
30           0.004196  
